In [1]:
import numpy as np
import os
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

distances = pd.read_feather("max_distance_of_50.feather")
df = pd.read_feather("prelim.feather").set_index("key")

In [2]:
keys = np.concatenate(distances["keys"].values)

df = df.loc[keys]

In [3]:
mins = df.groupby("sequence_key")[["easting","northing","City"]].min()
maxs = df.groupby("sequence_key")[["easting","northing"]].max()

maxs.columns = ["x_max","y_max"]
mins.columns = ["x_min","y_min","City"]
bounding_boxes = pd.concat([maxs,mins],axis=1)

In [8]:
matches = {}

for pos,i in enumerate(bounding_boxes.groupby("City")):

    print(f"Now Doing City {i[0]}, {len(i[1])} seqs, {n_citis - pos} left")
    val_df = i[1]
    seqs = val_df.index
    border = 1000

    for seq1 in tqdm.tqdm(seqs):
        
        x_min,x_max,y_min,y_max = val_df.loc[seq1,['x_min','x_max','y_min','y_max']]
        hits = []

        for seq2 in seqs:

            if seq1!=seq2:
                row = val_df.loc[seq2]


                x1 = any([x_min -border <= i  <= x_max + border for i in row[["x_max","x_min"]]])
                x2 = any([row["x_min"] -border <= i  <= row["x_max"] + border for i in [x_min,x_max]])

                y1 = any([y_min -border <= i  <= y_min + border for i in row[["y_min","y_min"]]])
                y2 = any([row["y_min"] -border <= i  <= row["y_min"] + border for i in [y_min,y_max]])

                if (x1 or x2) and (y1 or y2):

                    hits.append(seq2)

        if hits:
            matches[seq1] = [hits]

matches = pd.DataFrame.from_dict(matches,orient='index')
matches.columns = ["Mathes"]
matches.reset_index().to_feather(f"matches_with_{border}_border.feather")            

Now Doing City amman, 27 seqs, 24 left


100%|██████████| 27/27 [00:01<00:00, 23.20it/s]


Now Doing City amsterdam, 343 seqs, 23 left


100%|██████████| 343/343 [02:50<00:00,  2.01it/s]


Now Doing City austin, 336 seqs, 22 left


  5%|▌         | 18/336 [00:08<02:40,  1.98it/s]